In [133]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [134]:
data = pd.read_csv('train.csv', index_col = 'Id')
data.corr()['choose']

age                    0.017165
years_of_experience    0.029010
lesson_price          -0.134013
qualification          0.042160
physics                0.195183
chemistry              0.091878
biology                0.071310
english                0.022227
geography              0.006366
history               -0.004700
mean_exam_points       0.109409
choose                 1.000000
Name: choose, dtype: float64

In [135]:
data_X = data.iloc[:,:-1].to_numpy()
data_X
data_y = data.iloc[:,-1].to_numpy()


In [136]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [137]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {1:0, 0:0}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = classes[1]/len(self.labels)
#         prediction = max(classes, key=classes.get)
        
        return prediction

In [138]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1     # "impurity" - "нечистота", степень неопределенности
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [139]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [140]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [141]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 50

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    n_f = np.arange(1,n_features)
    n_features_r=np.random.choice(n_f,size=7,replace=False)
    
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        t_values_r= np.random.choice(t_values,size=int(len(t_values)*0.8), replace=True)
        for t in t_values_r:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [142]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, depth=1, depth_max = None, d=0):

    quality, t, index = find_best_split(data, labels)
#     print(quality, t, index)

    #  Базовый случай - прекращаем рекурсию, когда прирост качества меньше d или дерево достигло максимальной глубины
    #  или количество потенциальных листьев достигло максимума(что при данном алгоритме идентично глубине) или все элементы одного класса
    if quality <= d or depth>=depth_max:
#         print('leaf', depth, leaves)
        
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    depth +=1

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth, depth_max)
    false_branch = build_tree(false_data, false_labels, depth, depth_max)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [143]:
# Проход объекта по дереву для его классификации

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [144]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [145]:
def auc_(actual, predicted):
    S = pd.Series(actual, index=predicted)
    S.sort_index(inplace=True, ascending=False)
    tpr = [0]
    fpr = [0]
    for i in S: 
        tpr.append(tpr[-1]+(i==1)/S.sum())
        fpr.append(fpr[-1]+(i==0)/(len(S)-S.sum()))
    return np.trapz(tpr, x = fpr, dx=0.1)
#     return tpr,fpr
    

In [146]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(data_X, 
                                                                                     data_y, 
                                                                                     test_size = 0.3,stratify=data_X[:,3:6],
                                                                                     random_state =100)

In [147]:
np.random.seed(100)

In [148]:
# Построим дерево по всей выборке
my_tree = build_tree(data_X, data_y, depth_max = 30)

In [155]:
predict1 = predict(data_X,my_tree)

In [156]:
predict2 = predict(test_data,my_tree)

In [157]:
auc_(data_y,predict1)

0.8641161430202713

In [158]:
auc_(test_labels,predict2)

0.8775159874475514

In [159]:
data_test = pd.read_csv('test.csv', index_col = 'Id')

In [160]:
predict3 = predict(data_test.to_numpy(),my_tree)
pd_pred= pd.DataFrame({'Id':data_test.index, 'choose': predict3}, )
pd_pred.to_csv("submission.csv", index=False)